# Extract data from Prometheus measurements
Tests performed

Test 1: 17:53:47 - 17:56:15

Test 10: 17:34:13 - 17:36:39 last 17:38:06

Test 50: 18:57:59 - 18:59:41 last 19:02:05 actual stop 19:10

In [3]:
import pandas as pd

In [4]:
cpu_path = './data/grafana_data_export_cpu.csv'
ram_path = './data/grafana_data_export_ram.csv'
cluster_cpu_path = './data/grafana_data_export_cluster_cpu.csv'
cluster_ram_path = './data/grafana_data_export_cluster_ram.csv'

In [48]:
index = ['idle', '1', '10', '50']
columns = ['prediction-deployment-785855ff75-8c8rr']
idle_start = '2020-06-14T13:00:00+02:00'
idle_stop = '2020-06-14T14:00:00+02:00'
test_1_start = '2020-06-14T17:53:47+02:00'
test_1_stop = '2020-06-14T17:56:15+02:00'
test_10_start = '2020-06-14T17:34:13+02:00'
test_10_stop = '2020-06-14T17:38:06+02:00'
test_50_start = '2020-06-14T18:57:59+02:00'
test_50_stop = '2020-06-14T19:10:05+02:00'

tests = {
    'idle': {'start': '2020-06-14T13:00:00+02:00', 'stop': '2020-06-14T14:00:00+02:00' },
    '1': {'start': '2020-06-14T17:53:47+02:00', 'stop': '2020-06-14T17:56:15+02:00' },
    '10': {'start': }

In [6]:
# Return (CPU_avg, CPU_max, CPU_min)
def get_metrics(df, start, stop, columns):
    # Limit 
    range_df = df[df.index > start]
    range_df = range_df[range_df.index < stop]
    
    # Compute avg
    summary = range_df.describe()[columns]
    
    # Get CPU load
    return (summary.loc['mean'], summary.loc['max'], summary.loc['min'])

# Pod CPU

In [7]:
cpu_df = pd.read_csv(cpu_path, sep=';')
cpu_df.columns

Index(['Time', 'chronograf-chronograf-589555d744-htw2l',
       'coredns-d798c9dd-2ctbp', 'dashboard-metrics-scraper-c79c65bb7-q9xmb',
       'grafana-78fb9f87d4-drvjg', 'influxdb-0',
       'kafka-broker-5477466f54-bjmhz', 'kafka-zk-7d49659d74-gqct7',
       'kapacitor-kapacitor-7465969874-hglr5',
       'kubernetes-dashboard-6f89967466-vz68v',
       'local-path-provisioner-58fb86bdfd-9vtpd',
       'metrics-server-6d684c7b5-gljq4',
       'mqtt2kafka-deployment-8fd8bb6f8-677z4',
       'prediction-deployment-785855ff75-8c8rr',
       'prometheus-alertmanager-6f764c9487-qxgjs',
       'prometheus-kube-state-metrics-6756bbbb8-4nn8d',
       'prometheus-node-exporter-rhdq2', 'prometheus-node-exporter-s89ff',
       'prometheus-pushgateway-69c85cc8b-8lz6v',
       'prometheus-server-69b5694445-gbrv5', 'svclb-grafana-lhtwt',
       'svclb-grafana-tc7g8', 'svclb-traefik-576tq', 'svclb-traefik-vlcwc',
       'telegraf-84c4b5cdcb-l6tgw', 'traefik-65bccdc4bd-68bxv'],
      dtype='object')

In [8]:
# Set index to Time
cpu_df.set_index('Time', drop=True, inplace=True)

# Drop all NaN rows
cpu_df.dropna(axis=0, how='all', inplace=True)

In [9]:
cpu_idle = get_metrics(cpu_df, idle_start, idle_stop, columns)
cpu_1 = get_metrics(cpu_df, test_1_start, test_1_stop, columns)
cpu_10 = get_metrics(cpu_df, test_10_start, test_10_stop, columns)
cpu_50 = get_metrics(cpu_df, test_50_start, test_50_stop, columns)

In [57]:
list_cpu_idle  = list(map(lambda x: x.tolist()[0], cpu_idle))
list_cpu_1 = list(map(lambda x: x.tolist()[0], cpu_1))
list_cpu_10 = list(map(lambda x: x.tolist()[0], cpu_10))
list_cpu_50 = list(map(lambda x: x.tolist()[0], cpu_50))
cpu_final_df = pd.DataFrame([idle_list, list_1, list_10, list_50], columns=['CPU avg', 'CPU max', 'CPU min'])
cpu_final_df['index'] = index
cpu_final_df.set_index('index', inplace=True)
cpu_final_df

,CPU avg,CPU max,CPU min
index,,,
idle,0.016121,0.017,0.012
1,0.046222,0.085,0.018
10,0.272000,0.553,0.019
50,0.624327,0.804,0.021


# Pod RAM

In [161]:
ram_df = pd.read_csv(ram_path, sep=';', thousands=',')
ram_df

,Time,svclb-traefik-576tq,kafka-broker-5477466f54-bjmhz,prometheus-node-exporter-s89ff,prediction-deployment-785855ff75-8c8rr,local-path-provisioner-58fb86bdfd-9vtpd,prometheus-kube-state-metrics-6756bbbb8-4nn8d,prometheus-pushgateway-69c85cc8b-8lz6v,telegraf-84c4b5cdcb-l6tgw,influxdb-0,...,prometheus-node-exporter-rhdq2,kafka-zk-7d49659d74-gqct7,svclb-grafana-tc7g8,traefik-65bccdc4bd-68bxv,dashboard-metrics-scraper-c79c65bb7-q9xmb,svclb-grafana-lhtwt,prometheus-server-69b5694445-gbrv5,coredns-d798c9dd-2ctbp,chronograf-chronograf-589555d744-htw2l,kapacitor-kapacitor-7465969874-hglr5
0,2020-06-14T12:09:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-06-14T12:09:15+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-06-14T12:09:30+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-06-14T12:09:45+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-06-14T12:10:00+02:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,2020-06-14T20:28:15+02:00,4677632.0,728219648.0,6541312.0,406945792.0,8593408.0,9859072.0,6373376.0,32894976.0,74784768.0,...,10240000.0,52146176.0,1359872.0,14258176.0,19427328.0,1744896.0,205492224.0,8335360.0,29134848.0,33304576.0
1998,2020-06-14T20:28:30+02:00,4677632.0,728219648.0,6541312.0,406945792.0,8593408.0,9859072.0,6373376.0,32894976.0,74784768.0,...,10240000.0,52146176.0,1359872.0,14258176.0,19427328.0,1744896.0,205492224.0,8335360.0,29134848.0,33304576.0
1999,2020-06-14T20:28:45+02:00,4677632.0,728219648.0,6549504.0,406945792.0,8593408.0,9859072.0,6373376.0,32894976.0,74784768.0,...,10240000.0,52146176.0,1359872.0,14258176.0,19427328.0,1744896.0,205492224.0,8335360.0,29134848.0,33304576.0
2000,2020-06-14T20:29:00+02:00,4677632.0,728092672.0,6549504.0,406290432.0,8593408.0,9859072.0,6467584.0,32808960.0,74784768.0,...,10240000.0,52146176.0,1359872.0,14258176.0,19460096.0,1744896.0,212254720.0,8335360.0,29306880.0,33452032.0


In [162]:
# Set index to Time
ram_df.set_index('Time', drop=True, inplace=True)

# Drop all NaN rows
ram_df.dropna(axis=0, how='all', inplace=True)

In [177]:
ram_idle = get_metrics(ram_df, idle_start, idle_stop, columns)
ram_1 = get_metrics(ram_df, test_1_start, test_1_stop, columns)
ram_10 = get_metrics(ram_df, test_10_start, test_10_stop, columns)
ram_50 = get_metrics(ram_df, test_50_start, test_50_stop, columns)

(prediction-deployment-785855ff75-8c8rr    2.380749e+08
Name: mean, dtype: float64, prediction-deployment-785855ff75-8c8rr    239050752.0
Name: max, dtype: float64, prediction-deployment-785855ff75-8c8rr    237019136.0
Name: min, dtype: float64)
(prediction-deployment-785855ff75-8c8rr    4.003071e+08
Name: mean, dtype: float64, prediction-deployment-785855ff75-8c8rr    400379904.0
Name: max, dtype: float64, prediction-deployment-785855ff75-8c8rr    400183296.0
Name: min, dtype: float64)
(prediction-deployment-785855ff75-8c8rr    396046336.0
Name: mean, dtype: float64, prediction-deployment-785855ff75-8c8rr    400347136.0
Name: max, dtype: float64, prediction-deployment-785855ff75-8c8rr    393965568.0
Name: min, dtype: float64)
(prediction-deployment-785855ff75-8c8rr    4.075967e+08
Name: mean, dtype: float64, prediction-deployment-785855ff75-8c8rr    411381760.0
Name: max, dtype: float64, prediction-deployment-785855ff75-8c8rr    400650240.0
Name: min, dtype: float64)


# Cluster CPU

In [193]:
cluster_columns = ['k3smaster', 'k3sworker']

In [188]:
cluster_cpu_df = pd.read_csv(cluster_cpu_path, sep=';')
cluster_cpu_df

,Time,k3smaster,k3sworker
0,2020-06-14T11:37:00+02:00,NaN,NaN
1,2020-06-14T11:38:00+02:00,NaN,NaN
2,2020-06-14T11:39:00+02:00,NaN,NaN
3,2020-06-14T11:40:00+02:00,NaN,NaN
4,2020-06-14T11:41:00+02:00,NaN,NaN
...,...,...,...
662,2020-06-14T22:39:00+02:00,0.651,3.577
663,2020-06-14T22:40:00+02:00,0.633,3.620
664,2020-06-14T22:41:00+02:00,0.603,3.315
665,2020-06-14T22:42:00+02:00,0.580,2.133


In [189]:
cluster_cpu_df.set_index('Time', drop=True, inplace=True)
cluster_cpu_df.dropna(axis=0, how='all', inplace=True)
cluster_cpu_df

,k3smaster,k3sworker
Time,,
2020-06-14T12:27:00+02:00,0.171,0.055
2020-06-14T12:28:00+02:00,0.268,0.344
2020-06-14T12:29:00+02:00,0.371,0.483
2020-06-14T12:30:00+02:00,0.516,0.633
2020-06-14T12:31:00+02:00,0.517,0.727
...,...,...
2020-06-14T22:39:00+02:00,0.651,3.577
2020-06-14T22:40:00+02:00,0.633,3.620
2020-06-14T22:41:00+02:00,0.603,3.315


In [194]:
print(get_metrics(cluster_cpu_df, idle_start, idle_stop, cluster_columns))
print(get_metrics(cluster_cpu_df, test_1_start, test_1_stop, cluster_columns))
print(get_metrics(cluster_cpu_df, test_10_start, test_10_stop, cluster_columns))
print(get_metrics(cluster_cpu_df, test_50_start, test_50_stop, cluster_columns))

(k3smaster    0.495068
k3sworker    0.672712
Name: mean, dtype: float64, k3smaster    0.514
k3sworker    0.702
Name: max, dtype: float64, k3smaster    0.465
k3sworker    0.647
Name: min, dtype: float64)
(k3smaster    0.536333
k3sworker    0.928333
Name: mean, dtype: float64, k3smaster    0.545
k3sworker    1.116
Name: max, dtype: float64, k3smaster    0.531
k3sworker    0.748
Name: min, dtype: float64)
(k3smaster    0.51075
k3sworker    2.42300
Name: mean, dtype: float64, k3smaster    0.554
k3sworker    3.495
Name: max, dtype: float64, k3smaster    0.475
k3sworker    1.143
Name: min, dtype: float64)
(k3smaster    0.465385
k3sworker    3.882154
Name: mean, dtype: float64, k3smaster    0.582
k3sworker    4.975
Name: max, dtype: float64, k3smaster    0.404
k3sworker    0.825
Name: min, dtype: float64)


# Cluster RAM

In [200]:
cluster_ram_df = pd.read_csv(cluster_ram_path, sep=';', thousands=',')
cluster_ram_df

,Time,k3smaster,k3sworker
0,2020-06-14T12:21:00+02:00,NaN,NaN
1,2020-06-14T12:22:00+02:00,NaN,NaN
2,2020-06-14T12:23:00+02:00,NaN,NaN
3,2020-06-14T12:24:00+02:00,NaN,NaN
4,2020-06-14T12:25:00+02:00,NaN,NaN
...,...,...,...
677,2020-06-14T23:38:00+02:00,1.791967e+09,1.015203e+10
678,2020-06-14T23:39:00+02:00,1.791828e+09,1.015263e+10
679,2020-06-14T23:40:00+02:00,1.791791e+09,1.015119e+10
680,2020-06-14T23:41:00+02:00,1.791955e+09,1.015196e+10


In [201]:
# Set index to Time
cluster_ram_df.set_index('Time', drop=True, inplace=True)

# Drop all NaN rows
cluster_ram_df.dropna(axis=0, how='all', inplace=True)

In [202]:
print(get_metrics(cluster_ram_df, idle_start, idle_stop, cluster_columns))
print(get_metrics(cluster_ram_df, test_1_start, test_1_stop, cluster_columns))
print(get_metrics(cluster_ram_df, test_10_start, test_10_stop, cluster_columns))
print(get_metrics(cluster_ram_df, test_50_start, test_50_stop, cluster_columns))

(k3smaster    1.661405e+09
k3sworker    6.145399e+09
Name: mean, dtype: float64, k3smaster    1.672901e+09
k3sworker    6.189244e+09
Name: max, dtype: float64, k3smaster    1.649504e+09
k3sworker    6.114865e+09
Name: min, dtype: float64)
(k3smaster    1.738355e+09
k3sworker    9.547624e+09
Name: mean, dtype: float64, k3smaster    1.738437e+09
k3sworker    9.567498e+09
Name: max, dtype: float64, k3smaster    1.738256e+09
k3sworker    9.531142e+09
Name: min, dtype: float64)
(k3smaster    1.736748e+09
k3sworker    9.424284e+09
Name: mean, dtype: float64, k3smaster    1.737073e+09
k3sworker    9.457369e+09
Name: max, dtype: float64, k3smaster    1.736532e+09
k3sworker    9.388237e+09
Name: min, dtype: float64)
(k3smaster    1.748159e+09
k3sworker    9.780481e+09
Name: mean, dtype: float64, k3smaster    1.756967e+09
k3sworker    9.916613e+09
Name: max, dtype: float64, k3smaster    1.746383e+09
k3sworker    9.702498e+09
Name: min, dtype: float64)
